In [1]:
import sys
import os
sys.path.append("/Users/baeyeongmin/Desktop/workspace/dacon-2024-gbt-hackerton")
sys.path.append("/root/dacon-2024-gbt-hackerton")

In [14]:
import os
import wandb
import torch
import pandas as pd
from src.config import CFG
from src.data_processing import load_data, preprocess_data, get_label_encoded, split_data
from src.models import get_model_and_tokenizer
from src.datasets import TextDataset
from src.training import train_model
from src.inference import get_test_predictions
from src import root_path
from src.utils import get_sample_submission
from torch.utils.data import DataLoader

CFG.model_type = "primary"
wrun = wandb.init(project="dacon-gbt-2024-hackerton", config=CFG)

# Data loading and preprocessing
train_df, test_df = load_data(root_path=root_path)
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

train_df, label_encoder = get_label_encoded(train_df, model_type=CFG.model_type)
train_df, val_df = split_data(train_df)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ryanbae. Use `wandb login --relogin` to force relogin


In [8]:
# Model and tokenizer initialization
model, tokenizer = get_model_and_tokenizer(CFG.model_name, len(label_encoder), CFG.attention_type)

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Dataset and DataLoader creation
train_dataset = TextDataset(train_df.text.tolist(), train_df.label.tolist(), tokenizer, max_len=CFG.tokenizer_max_len)
val_dataset = TextDataset(val_df.text.to_list(), val_df.label.tolist(), tokenizer, max_len=CFG.tokenizer_max_len)
test_dataset = TextDataset(test_df.text.tolist(), None, tokenizer, max_len=CFG.tokenizer_max_len)

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers, pin_memory=CFG.pin_memory)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers, pin_memory=CFG.pin_memory)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers, pin_memory=CFG.pin_memory)

# Model training
trained_model = train_model(model, train_loader, val_loader, CFG)

# Inference
label_decoder = {i: label for label, i in label_encoder.items()}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictions = get_test_predictions(trained_model, test_loader, device, label_decoder)


Validating: 100%|██████████| 171/171 [00:35<00:00,  4.86it/s]


Error: You must call wandb.init() before wandb.log()